In [1]:
"""
Wrapper methods around "import csv" that
are very specific to p3.
Created by Uki D. Lucas on Feb. 4, 2017
"""

'\nWrapper methods around "import csv" that\nare very specific to p3.\nCreated by Uki D. Lucas on Feb. 4, 2017\n'

In [2]:
# snapping actual values to given labels

def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return array[idx]

# TEST
#assert (find_nearest(steering_labels, -0.951) == -1.00),"method find_nearest() has problem"

In [3]:

import numpy as np

# TODO implement batch_from, batch_to - I did not need it for 8037 rows
# TODO implement has_header_row
def read_csv(file_path):
    """
    Usage:
    headers, data = read_csv(file_path)
    Parameter: 
    - file_path: can be relative path "../../../DATA/stuff.csv"
    Returns:
    - headers: array of strings e.g. ['steering', 'throttle', 'brake', 'speed']
    - data: array of strings, you have to convert values to int, float yourself
   test_read_csv()
    """
    import csv
    # Opening spreadsheet to read in TEXT mode: 'rt'
    with open(file_path, 'rt') as csvfile:
        # Most common format of CSV, TODO improve
        payload = csv.reader(csvfile, delimiter=',', quotechar='"') 
        row_counter = 0
        headers = []
        data = []
        for row in payload:
            row_counter = row_counter + 1
            
            if row_counter == 1:
                headers = row
                # print (type(row))
                # print ( row)
                # print ('\t '.join(row))
            elif 1 < row_counter < 3:
                # print (type(row))
                # print ( row)
                # print ('\t '.join(row))
                data.append(row)
            else:
                data.append(row)
        print("imported rows", row_counter)
        # I am returning data as numpy array instead of list
        # because it is handier to use it.
        return headers, np.array(data)
    
    
    
def test_read_csv():
    """
    This test is specific to Uki's enviroment.
    """
    data_dir = "../../../DATA/behavioral_cloning_data/"
    headers, data = read_csv(file_path = data_dir + 'driving_log.csv')
    print("headers \n",headers)
    print("3rd row of data \n",data[2:3])
# test_read_csv()

In [4]:
import numpy as np
import math
    
def split_random(matrix, percent_train=70, percent_test=15):
    """
    Splits matrix data into randomly ordered sets 
    grouped by provided percentages.
    
    Usage:
    rows = 100
    columns = 2
    matrix = np.random.rand(rows, columns)
    training, testing, validation = \
    split_random(matrix, percent_train=80, percent_test=10)
    
    percent_validation 10
    training (80, 2)
    testing (10, 2)
    validation (10, 2)
    
    Returns:
    - training_data: percentage_train e.g. 70%
    - testing_data: percent_test e.g. 15%
    - validation_data: reminder from 100% e.g. 15%
    Created by Uki D. Lucas on Feb. 4, 2017
    """
    #print(matrix)  
    row_count = matrix.shape[0]
    np.random.shuffle(matrix)
    
    end_training = int(math.ceil(row_count*percent_train/100))    
    end_testing = end_training + int(math.ceil((row_count * percent_test/100)))
    
    percent_validation = 100 - percent_train - percent_test
    
    training = matrix[:end_training]
    testing = []
    validation = []
    
    if percent_validation < 0:
        print("Make sure that the provided sum of " + \
        "training and testing percentages is equal, " + \
        "or less than 100%.")
        
        testing = matrix[end_training:]
    else:
        print("percent_validation", percent_validation)
        
        testing = matrix[end_training:end_testing]
        validation = matrix[end_testing:]
    
    return training, testing, validation

# TEST:

def test_split_random():
    rows = 8037
    columns = 2
    matrix = np.random.rand(rows, columns)
    training, testing, validation = split_random(matrix, percent_train=80, percent_test=20) 

    print("training",training.shape)
    print("testing",testing.shape)
    print("validation",validation.shape)
    
    print("sum",training.shape[0] + testing.shape[0])
    
    #print(split_random.__doc__)
# test_split_random()

In [5]:
def get_image_center_values(matrix):
    data = [row[0] for row in matrix]
    return np.array(data)

def get_image_left_values(matrix):
    data = [row[1] for row in matrix]
    return np.array(data)

def get_image_right_values(matrix):
    data = [row[2] for row in matrix]
    return np.array(data)

def get_steering_values(matrix):
    data = [float(row[3]) for row in matrix]
    return np.array(data).astype('float32')

def get_throttle_values(matrix):
    data = [float(row[4]) for row in matrix]
    return np.array(data)

def get_brake_values(matrix):
    data = [float(row[5]) for row in matrix]
    return np.array(data)

def get_speed_values(matrix):
    data = [float(row[6]) for row in matrix]
    return np.array(data)

In [7]:
# for custom metrics
import keras.backend as K

def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

def false_rates(y_true, y_pred):
    false_neg = ...
    false_pos = ...
    return {
        'false_neg': false_neg,
        'false_pos': false_pos,
    }

Using TensorFlow backend.


In [9]:
def sort_unique_floats(array_x):
    # assure that the array is numpy and numerical
    #array_x = array_x.astype(np.float)
    
    # get unique values, a.k.a. set of values
    labels_set = set(array_x)
    #print("labels_set\n", labels_set)
    
    # set is not sorted, so convert it to a numpy array
    unique_labels = np.array(list(labels_set))
    #print("unique_labels\n", unique_labels.shape, unique_labels)
    
    sorted_unique_labels = np.sort(unique_labels)
    #print("sorted_unique_labels\n", sorted_unique_labels.shape, sorted_unique_labels)
    return sorted_unique_labels
    
# TEST   


In [10]:
def encode_one_hot(all_possible_classes, training_labels):
    """
    Creates one hot encoded vector from a list {1D vector (None,)} containing training labels.
    - find all unique labels
    - count all unique labels
    - create a zero filed array, size equal to count of all unique labels
    - order the unique values (small to large)
    - create empty output matrix
    - for each sample's label create zero vector and set one in position of that label
    Created by Uki D. Lucas
    """
    all_possible_classes = sort_unique_floats(all_possible_classes)
    
    # possible float rounding errors
    all_possible_classes = np.round_(all_possible_classes, decimals=1)
    print("all_possible_classes\n", all_possible_classes)
    
    class_count = len(all_possible_classes)
    print("class_count:", class_count)
    
    sample_count = len(training_labels)
    print("sample_count:", sample_count)
    
    one_hot = np.zeros(shape=(sample_count, class_count), dtype=np.int)
    print("one_hot shape", one_hot.shape)
    #print("one_hot 3 shape", one_hot[3].shape)
    #print("one_hot 3", one_hot[3])
     
    for i in range( sample_count ): 
        actual_label = float(training_labels[i])
        #print("looking for actual_label:", actual_label)
        
        # find first index of actual_label
        item_index = np.where(all_possible_classes == actual_label)[0]
        #print("found", item_index)
        one_hot[i][item_index] = 1
        #print("one_hot[i]", one_hot[i])
    print("one_hot[0] \n", one_hot[0])
    return one_hot

In [11]:
def show_layers(model):
    for i in range(len(model.layers)):
        layer = model.layers[i]
        print(i, ") ",layer.name, "\t\t is trainable: ", layer.trainable)
        #layer.trainable = False
    return len(model.layers)

In [12]:
import numpy as np
import random
import math
from matplotlib import pyplot as plt

def margin(value):
    return value*(5/100)

def plot_histogram(name, values, change_step):
    
    min_value = min(values)
    print("min_value", min_value)
    max_value = max(values)
    print("max_value", max_value)
    
    spread = max_value-min_value
    print("spread", spread)
    recommended_bins = math.ceil(spread/change_step)
    print("recommended number of classes", recommended_bins)
    
    bins = np.linspace(math.floor(min(values)), 
                       math.ceil(max(values)),
                       recommended_bins)

    plt.xlim([
        min_value - margin(min_value), 
        max_value + margin(max_value)])

    plt.hist(values, bins=bins, alpha=0.5)
    plt.title('Distribution of ' + name)
    plt.xlabel('values')
    plt.ylabel('occurance')

    plt.show()